**排他的論理和（ExOR）の学習と学習結果の表示**

学習用データの読み込み

In [3]:
from google.colab import files
uploaded = files.upload()

Saving trainingdata.txt to trainingdata (1).txt


排他的論理和の学習

In [0]:
# -*- coding: utf-8 -*-
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
import chainer.initializers as I
from chainer import training, cuda, optimizers, Variable
from chainer.training import extensions

class LogicCircuit(chainer.Chain):
    def __init__(self):
        w = I.Normal(scale=1.0) # モデルパラメータの初期化（平均0，分散1の分布に従う）
        super(LogicCircuit, self).__init__(
            l1 = L.Linear(None, 4, initialW=w, initial_bias=0.5),
            l2 = L.Linear(None, 8, initialW=w, initial_bias=0.5),
            l3 = L.Linear(None, 2, initialW=w, initial_bias=0.5),
        )

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)        
        return y        

def main():

    epoch = 300
    batchsize = 8

    # ニューラルネットワークの登録
    model = L.Classifier(LogicCircuit(), lossfun=F.softmax_cross_entropy)
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)
    
    gpu_device = 0
    cuda.get_device(gpu_device).use()
    model.to_gpu(gpu_device)
    np = cuda.cupy
    
    # データの作成
    #trainx = np.array(([0,0], [0,1], [1,0], [1,1], [0.01,0.01], [0.01,1.01], [1.01,0.01], [1.01,1.01]), dtype=np.float32)
    #trainy = np.array(([0, 1, 1, 0, 0, 1, 1, 0]), dtype=np.int32)
    #train = chainer.datasets.TupleDataset(trainx, trainy)
    #test = chainer.datasets.TupleDataset(trainx, trainy)

    with open('trainingdata.txt', 'r') as f:
      lines = f.readlines()
      
    data = []
    for l in lines:
      d = l.strip().split()
#      data.append(map(float, d))
      data.append(d)

    data = np.array(data, dtype=np.float32)
    print(data)
    trainx, trainy = np.hsplit(data, [2])
    trainy= trainy[:, 0]
    trainx= np.array(trainx, dtype=np.float32)
    print(trainx)
    trainy= np.array(trainy, dtype=np.int32)
    print(trainy)
    train = chainer.datasets.TupleDataset(trainx, trainy)
    test = chainer.datasets.TupleDataset(trainx, trainy)
    
    
    # ニューラルネットワークの登録
#    model = L.Classifier(LogicCircuit(), lossfun=F.softmax_cross_entropy)
#    optimizer = chainer.optimizers.Adam()
#    optimizer.setup(model)
    

    # イテレータの定義
    train_iter = chainer.iterators.SerialIterator(train, batchsize)# 学習用
    test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)# 評価用

    # アップデータの登録
    # アップデータはミニバッチ毎のデータから求めた損失値を使って，誤差逆伝播とパラメータの更新を行う機構
    updater = training.StandardUpdater(train_iter, optimizer)
    # トレーナーの登録
    # トレーナーは学習を実行するところ
    # 学習とは，1．入力データが各層を伝搬していき出力値を計算，2．出力値と教師データから損失（誤差）を計算，3．誤差を逆伝播させ誤差が小さくなるように，パラメータを更新，という一連の流れを言う
    # 1データ毎にこれをやると効率が悪いので，ある程度のデータの塊（これをミニバッチ）で行う．例えばミニバッチサイズ100だと，100個のデータの誤差の平均値を基にパラメータを更新する．
    trainer = training.Trainer(updater, (epoch, 'epoch'))

    # 学習状況の表示や保存
    trainer.extend(extensions.Evaluator(test_iter, model))# エポック数の表示
    trainer.extend(extensions.dump_graph('main/loss'))#ニューラルネットワークの構造
    trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], 'epoch',file_name='loss.png'))#誤差のグラフ
    trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'],'epoch', file_name='accuracy.png'))#精度のグラフ
    trainer.extend(extensions.LogReport(), trigger=(10, 'epoch'))#ログ
#    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss','main/accuracy', 'validation/main/accuracy', 'elapsed_time'] ), trigger=(10, 'epoch'))#計算状態の表示
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss','main/accuracy', 'elapsed_time'] ), trigger=(10, 'epoch'))#計算状態の表示
    trainer.extend(extensions.snapshot(), trigger=(100, 'epoch'))# エポック毎にトレーナーの状態（モデルパラメータも含む）を保存する（なんらかの要因で途中で計算がとまっても再開できるように）

#    chainer.serializers.load_npz("result/snapshot_iter_500", trainer)

    # 学習開始
    trainer.run()

    # 途中状態の保存
    chainer.serializers.save_npz("result/ExOR.model", model)

    # 学習結果の評価

    print("\n")    
    testx = np.array(([0,0], [0,1], [1,0], [1,1]), dtype=np.float32)
    
    for i in range(len(testx)):
        x = chainer.Variable(testx[i].reshape(1,2))
        result = F.softmax(model.predictor(x))
        print("input: {}, result: {}".format(testx[i], result.data.argmax()))

if __name__ == '__main__':
    main()


テストデータ（入力の順番入れ替え）の読み込み

In [0]:
uploaded = files.upload()

読み込んだテストデータ（入力の順番入れ替え）に対する結果の表示

In [0]:
    with open('testdata1.txt', 'r') as f:
      lines = f.readlines()
      
    data = []
    for l in lines:
      d = l.strip().split()
      data.append(d)

    testx= np.array(data, dtype=np.float32)
    print(testx)
    
    model = L.Classifier(LogicCircuit(), lossfun=F.softmax_cross_entropy)
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    
    chainer.serializers.load_npz("result/ExOR.model", model)

    for i in range(len(testx)):
        x = chainer.Variable(testx[i].reshape(1,2))
        result = F.softmax(model.predictor(x))
        print("input: {}, result: {}".format(testx[i], result.data.argmax()))


テストデータ（学習データ＋ノイズ0.01）の読み込み

In [0]:
uploaded = files.upload()

In [1]:
    with open('testdata2.txt', 'r') as f:
      lines = f.readlines()
      
    data = []
    for l in lines:
      d = l.strip().split()
      data.append(d)

    testx= np.array(data, dtype=np.float32)
    print(testx)
    
    model = L.Classifier(LogicCircuit(), lossfun=F.softmax_cross_entropy)
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    
    chainer.serializers.load_npz("result/ExOR.model", model)

    for i in range(len(testx)):
        x = chainer.Variable(testx[i].reshape(1,2))
        result = F.softmax(model.predictor(x))
        print("input: {}, result: {}".format(testx[i], result.data.argmax()))


FileNotFoundError: ignored

In [0]:
!ls -l result

学習回数による損失関数の変化

In [0]:
from IPython.display import Image,display_png
display_png(Image('result/loss.png'))

学習回数による正解率の変化

In [0]:
from IPython.display import Image,display_png
display_png(Image('result/accuracy.png'))

ネットワーク構造の表示

In [0]:
!dot -T png result/cg.dot -o result/cg.png

from IPython.display import Image,display_png

display_png(Image('result/cg.png'))